1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고,
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1500, # 하나의 chunk 가 가질수 있는 토큰 수 
    chunk_overlap=200, # 겹치는 부분 -> 유사도 검색을 할 때 우리가 원하는 문서를 가지고 오는 확률을 높이기 위함함
)

loader = Docx2txtLoader("../tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large')

In [ ]:
import os
from langchain_pinecone import PineconeVectorStore

index_name = 'upstage-embedding'
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(documents=document_list, embedding=embedding, index_name=index_name)


In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [13]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/data/anaconda3/envs/langchain2/lib/python3.10/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt":prompt}
)

In [15]:
query = '연봉 5000만원인 직장인의 소득세는 얼마인가요?'

In [16]:
ai_message = qa_chain({"query":query})

/tmp/ipykernel_3136323/980961933.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query":query})


In [17]:
ai_message

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5000만원 직장인의 소득세는 여러 요소에 따라 달라질 수 있지만, 기본적으로 소득세율을 적용하여 계산합니다. 제공된 정보에서는 구체적인 소득세 계산에 필요한 세율이나 기타 공제 사항이 명시되어 있지 않아 정확한 금액을 제시할 수 없습니다. 따라서 소득세에 대한 구체적인 계산은 추가적인 정보가 필요합니다.'}

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5000만원 직장인의 소득세는 여러 요소에 따라 달라질 수 있지만, 기본적으로 소득세율을 적용하여 계산합니다. 제공된 정보에서는 구체적인 소득세 계산에 필요한 세율이나 기타 공제 사항이 명시되어 있지 않아 정확한 금액을 제시할 수 없습니다. 따라서 소득세에 대한 구체적인 계산은 추가적인 정보가 필요합니다.'}